In [ ]:
import ruleminer
import logging
import sys
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
logging.basicConfig(stream=sys.stdout, 
                    format='%(asctime)s %(message)s',
                    level=logging.WARNING)

# Decision tree for iris dataset

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
df = pd.DataFrame(
    columns = [c.replace(")", "").replace("(", "") for c in iris['feature_names']], 
    data = iris['data']
)
df['target'] = [iris['target_names'][x] for x in iris['target']]
df[['target']] = OrdinalEncoder(dtype=int).fit_transform(df[['target']])

In [ ]:
ensemble_expressions = ruleminer.fit_ensemble_and_extract_expressions(df, "target", max_depth = 2)

In [ ]:
# finding the best rules in the ensemble
sum_support = 0
params = {"filter": {'confidence': 0.0, 'abs support': 0},
          "decimal": 1}
for expressions in ensemble_expressions:
    templates = [{'expression': expression} for expression in expressions]
    miner = ruleminer.RuleMiner(templates=templates, 
                            data=df, params=params)
    if miner.rules['abs support'].sum() > sum_support:
        sum_support = miner.rules['abs support'].sum()
        best = miner

In [ ]:
best.rules

In [ ]:
best.rules['abs support'].sum()

# Decision tree with insurance data

In [ ]:
df = pd.DataFrame(
    columns=[
        "Name",
        "Type",
        "Assets",
        "TV-life",
        "TV-nonlife",
        "Own funds",
        "Excess",
    ],
    data=[
        ["Insurer1", "life insurer", 1000.0, 800.0, 0.0, 200.0, 200.0],
        ["Insurer2", "non-life insurer", 4000.0, 0.0, 3200.0, 800.0, 800.0],
        ["Insurer3", "non-life insurer", 800.0, 0.0, 700.0, 100.0, 100.0],
        ["Insurer4", "life insurer", 2500.0, 1800.0, 0.0, 700.0, 700.0],
        ["Insurer5", "non-life insurer", 2100.0, 0.0, 2200.0, 200.0, 200.0],
        ["Insurer6", "life insurer", 9000.0, 8700.0, 0.0, 300.0, 200.0],
        ["Insurer7", "life insurer", 9000.0, 8800.0, 0.0, 200.0, 200.0],
        ["Insurer8", "life insurer", 9000.0, 8900.0, 0.0, 100.0, 200.0],
        ["Insurer9", "non-life insurer", 9000.0, 8850.0, 0.0, 150.0, 200.0],
        ["Insurer10", "non-life insurer", 9000.0, 0, 8750.0, 250.0, 199.99],
    ],
)
df.index.name="id"
for i in range(0, 0):
    df = df.append(df, ignore_index=True)
    
df[['Type']] = OrdinalEncoder(dtype=int).fit_transform(df[['Type']])
df[['Name']] = OrdinalEncoder(dtype=int).fit_transform(df[['Name']])

In [ ]:
expressions = ruleminer.fit_dataframe_to_ensemble(df, max_depth = 1)

In [ ]:
templates = [{'expression': solution} for solution in expressions]
params = {"filter": {'confidence': 0.75, 'abs support': 2},
          "metrics": ['confidence', 'abs support', 'abs exceptions']}
r = ruleminer.RuleMiner(templates=templates, data=df, params = params)

In [ ]:
r.rules

In [ ]:
df_r = r.evaluate()

In [ ]:
df_r[df_r['result']==False]